In [ ]:
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/accelerate-0.29.3-py3-none-any.whl'
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/aiohttp-3.9.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl'
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/fsspec-2024.3.1-py3-none-any.whl'
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl'
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/unsloth-2024.4-py3-none-any.whl'
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/peft-0.10.0-py3-none-any.whl'


In [ ]:

!pip install '/kaggle/input/llm-detect-pip/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_nvjitlink_cu12-12.3.52-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl'
##!pip install '/kaggle/input/llm-detect-pip/nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/llm-detect-pip/nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl'
!pip install '/kaggle/input/nvidia-nccl-2-19-3/nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl'
#!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/triton-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'



In [ ]:
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/shtab-1.7.1-py3-none-any.whl'
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/tyro-0.8.3-py3-none-any.whl'
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/trl-0.8.5-py3-none-any.whl'

In [ ]:
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl'

In [ ]:
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl'

In [ ]:
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl'

In [ ]:
!pip install '/kaggle/input/easy-to-use-unslot-wheel/pip-wheel/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl'

In [ ]:
import torch
print(f"pytorch version {torch.__version__}")

In [ ]:
from unsloth import FastLanguageModel

In [ ]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from peft import PeftModel
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [ ]:
test = pd.read_csv(open('/kaggle/input/lmsys-chatbot-arena/test.csv'))

In [ ]:
def generate_test_prompt(data_point):
    return f"""Analyze the conversation between two chatbots (model_a and model_b) and their corresponding responses (response_a and response_b) to a given prompt. Determine which model provided the more preferred 
response based on the human preference label (Preference). Return the predicted preference as one of three labels: 'winner_model_a', 'winner_model_b', or 'winner_tie', along with the logits for each label.

Prompt: {data_point["prompt"]}
Model A Response: {data_point["response_a"]}
Model B Response: {data_point["response_b"]}
"""

In [ ]:
'''
MODEL_PATH = "/kaggle/input/llama-3/transformers/8b-hf/1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model1 = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config=quantization_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, dtype=torch.float16)
'''

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.float16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",

    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit"
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/kaggle/input/unslothmistral-7b-instruct-v0-2-bnb-4bit/unsloth-mistral-7b-instruct-v0.2-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
submission = pd.read_csv(open('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv'))
test = pd.read_csv(open('/kaggle/input/lmsys-chatbot-arena/test.csv'))
train = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")

In [ ]:
result = []
for index, row in train.iterrows():
    if(row['winner_model_a']==1): result.append('winner_model_a')
    elif(row['winner_model_b']==1): result.append('winner_model_b')
    elif(row['winner_tie']==1): result.append('winner_tie')
train.insert(6, "Preference", result, True)
train

In [ ]:
train = train[:1000]
train.shape

In [ ]:
train_size = int(train.shape[0]*0.6)
test_val_size = train.shape[0]-train_size
train_, test_val  = train_test_split(train, train_size=train_size,
                                    test_size=test_val_size,
                                    random_state=42)

test_, val_ = train_test_split(test_val, train_size=int(test_val_size/2),
                                    test_size=int(test_val_size/2),
                                    random_state=42)

In [ ]:
def past_generate_prompt(data_point):
    return f"""
            Analyze the prompt and responses(response_a, response_b) from two chatbots(model_a, model_b).
            Then predict the human preference of those responses- if it is "winner_model_a", "winner_model_b" or
            "winner_tie". Return the answer as the correspoding preference label "winner_model_a", "winner_model_b" or
            "winner_tie" and the logits for each label in the order of preference.
            ----------------------------------------------------------------------------------------------------------
            prompt: {data_point["prompt"]}
            ----------------------------------------------------------------------------------------------------------
            model_a: {data_point["model_a"]}
            response_a: {data_point["response_a"]}
            ----------------------------------------------------------------------------------------------------------
            model_b: {data_point["model_b"]}
            response_b: {data_point["response_b"]}
            ----------------------------------------------------------------------------------------------------------
            Preference= {data_point["Preference"]} """.strip()

def generate_prompt(data_point):
    return f"""Analyze the conversation between two chatbots (model_a and model_b) and their corresponding responses (response_a and response_b) to a given prompt. Determine which model provided the more preferred
response based on the human preference label (Preference). Return the predicted preference as one of three labels: 'winner_model_a', 'winner_model_b', or 'winner_tie', along with the logits for each label.

Prompt: {data_point["prompt"]}
Model A Response: {data_point["response_a"]}
Model B Response: {data_point["response_b"]}
Human Preference Label: {data_point["Preference"]}
"""

def past_generate_test_prompt(data_point):
    return f"""
            Analyze the prompt and responses(response_a, response_b) from two chatbots(model_a, model_b).
            Then predict the human preference of those responses- if it is "winner_model_a", "winner_model_b" or
            "winner_tie". Return the answer as the correspoding preference label "winner_model_a", "winner_model_b" or
            "winner_tie" and the logits for each label in the order of preference.
            ----------------------------------------------------------------------------------------------------------
            prompt: {data_point["prompt"]}
            ----------------------------------------------------------------------------------------------------------

            response_a: {data_point["response_a"]}
            ----------------------------------------------------------------------------------------------------------

            response_b: {data_point["response_b"]}
            ----------------------------------------------------------------------------------------------------------
            Preference: """.strip()

def generate_test_prompt(data_point):
    return f"""Analyze the conversation between two chatbots (model_a and model_b) and their corresponding responses (response_a and response_b) to a given prompt. Determine which model provided the more preferred
response based on the human preference label (Preference). Return the predicted preference as one of three labels: 'winner_model_a', 'winner_model_b', or 'winner_tie', along with the logits for each label.

Prompt: {data_point["prompt"]}
Model A Response: {data_point["response_a"]}
Model B Response: {data_point["response_b"]}
"""


In [ ]:
X_train = pd.DataFrame(train_.apply(generate_prompt, axis=1),
                       columns=["text"])
X_eval = pd.DataFrame(val_.apply(generate_prompt, axis=1),
                      columns=["text"])
X_test = pd.DataFrame(test_.apply(generate_test_prompt, axis=1), columns=["text"])

y_true = test_.Preference

train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

In [ ]:
def evaluate(y_true, y_pred):
    labels = ['winner_model_a', 'winner_model_b', 'winner_tie']
    mapping = {'none':0, 'winner_model_a': 1, 'winner_model_b': 2, 'winner_tie': 3}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_data,
    eval_dataset = eval_data,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        #evaluation_strategy="steps",         # evaluate at the end of each epoch
        #save_strategy="steps",
        #save_steps=10,                      # save checkpoint every 500 steps
        #eval_steps=10,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = True,
        bf16 = False,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        #load_best_model_at_end=True,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
import wandb
wandb.init(mode='disabled')

In [ ]:
import gc
gc.collect()

In [ ]:
trainer.train()

In [ ]:
test['text'] = test.apply(lambda x: generate_test_prompt(x), axis=1)

In [ ]:
'''
prompt = test.loc[2]["text"]

#print(prompt)

#FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(prompt, return_tensors='pt').to(device)

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
out1 = tokenizer.batch_decode(outputs)
print("*************")
print(out1)
'''

In [ ]:
def extract_logits(text):
    try:
        d = {'winner_a': [], 'winner_b': [], 'winner_tie': []}
        logits = re.findall(r"Logits: \[(.*?)\]", text)[0]
        logits_dict = {k.strip(): float(v) for k, v in [item.split(": ") for item in logits[1:-1].split(", ")]}

        for k, v in logits_dict.items():
            if '_a' in k:
                d['winner_a'].append(v)
            elif '_b' in k:
                d['winner_b'].append(v)
            elif '_tie' in k:
                d['winner_tie'].append(v)
            
        df = pd.DataFrame.from_dict(d)
    
    except:
        d = {'winner_a': [0.33], 'winner_b': [0.33], 'winner_tie': [0.33]}
        df = pd.DataFrame.from_dict(d)


    
    

    return df

In [ ]:
prompt = X_test.iloc[2]["text"]

#print(prompt)



FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

#print(inputs)

print("*************")
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
out1 = tokenizer.batch_decode(outputs)

print(out1)



In [ ]:
# import re
# label_pattern = re.compile(r'Human Preference Label:\s*(\w+)')

In [ ]:
# def get_logits(text):
#     logits_pattern = re.compile(r'Logits:\s*\[(.*?)\]')
#     logits_match = logits_pattern.search(text[0])

#     if logits_match:
#         logits_str = logits_match.group(1)
#         logits_list = [(x.strip()) for x in logits_str.split(',')]

#         # Output results
#         #print("Logits:", logits_list)
#     else:
#         # if it is not able to find string, give deafult values
#         logits_list = ["'winner_model_a': 0.33", "'winner_model_b': 0.33", "'winner_tie': 0.33"]
    
    
    
#     return logits_list

In [ ]:
# y_pred = []
# logits = pd.DataFrame()
# for i in tqdm(range(len(X_test))):
#     prompt = X_test.iloc[i]["text"]
#     FastLanguageModel.for_inference(model) # Enable native 2x faster inference

#     inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

#     outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
#     out1 = tokenizer.batch_decode(outputs)
#     try:
#         preferences = label_pattern.search(out1[0]).group(1)
#         lgts = extract_logits(out1[0])
#     except Exception as e:
#         print(out1)
#         preferences = 'winner_tie'
#     print(preferences)
#     print(lgts)
#     y_pred.append(preferences)
    
#     logits = pd.concat([logits, lgts],axis=0)

In [ ]:
##evaluate(y_true, y_pred)

In [ ]:
# def get_logits(text):
#     logits_pattern = re.compile(r'Logits:\s*\[(.*?)\]')
#     logits_match = logits_pattern.search(text[0])

#     if logits_match:
#         logits_str = logits_match.group(1)
#         logits_list = [(x.strip()) for x in logits_str.split(',')]

#         # Output results
#         #print("Logits:", logits_list)
#     else:
#         # if it is not able to find string, give deafult values
#         logits_list = ["'winner_model_a': 0.33", "'winner_model_b': 0.33", "'winner_tie': 0.33"]
    
    
    
#     return logits_list

In [ ]:
logits_list = pd.DataFrame()
for i in tqdm(range(len(test))):
    prompt = test.iloc[i]["text"]
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

    inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    out1 = tokenizer.batch_decode(outputs)
    preferences = extract_logits(out1[0])
    #print(preferences)
    logits_list = pd.concat([logits_list, preferences],axis=0)

In [ ]:
# import pandas as pd



# # Function to convert string list to dictionary
# def logits_to_dict(logits):
#     return {item.split(':')[0].strip("' "): float(item.split(':')[1].strip()) for item in logits}

# # Convert each list to a dictionary
# logits_dicts = [logits_to_dict(logits) for logits in logits_list]

# # Create DataFrame
# df = pd.DataFrame(logits_dicts)

# df


In [ ]:
df1 = logits_list.reset_index(drop=True)

In [ ]:
df = pd.concat([test['id'], df1],axis=1)

In [ ]:
df

In [ ]:
df.to_csv('submission.csv', index=False)